# Deep Learning Model for Character Recognition

In [ ]:
# libraries used for the model building

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D
from tensorflow.keras import optimizers
import datetime
import pickle


print(tf.__version__)

import numpy as np
import cv2
import matplotlib.pyplot as plt



In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1)
train_generator = train_datagen.flow_from_directory(
        '/content/data/train',  # this is the target directory. It contains the different classes of the characters.
        target_size=(28,28),  # all images will be resized to 28x28
        batch_size=1,
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        '/content/data/val',  # this is the target directory
        target_size=(28,28),  # all images will be resized to 28x28        batch_size=1,
        class_mode='categorical')

In [ ]:
Folders architecture

______ data/train :
    _________Class_0:
        __________img1.jpg //28*28 pixels
        ...
        __________imgN.jpg // 28*28 pixels
    _________Class_1
    ...
    _________Class_9
    _________Class_TN

______ data/test :
    _________Class_0
    _________Class_1
    ...
    _________Class_9
    _________Class_TN


In [ ]:
model = Sequential()
model.add(Conv2D(32, (24,24), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (20,20), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (20,20), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(11, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.00001), metrics=['accuracy'])

In [ ]:
class stop_training_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc') > 0.992):
      self.model.stop_training = True

In [ ]:
batch_size = 1
model.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples // batch_size,
      validation_data = validation_generator,
      validation_steps = validation_generator.samples // batch_size,
      epochs = 80)

In [ ]:
#_, accuracy = model.evaluate
test_loss, test_acc = model.evaluate(train_generator, validation_generator, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
# # save model and architecture to single file
model.save("ocrmodel.h5")
print("Saved model to disk")

In [ ]:
model = tf.keras.models.load_model('ocrmodel.h5')

In [ ]:
model.summary()